Example shows how to find a minimum using computation graph and optimiser.

See `HelloScalaTensorflow2`

In [1]:
//workaround to load tensorflow_2.11-0.1.1-SNAPSHOT-linux-cpu-x86_64.jar
new {
    val userHome = System.getProperty("user.home")
    val pathPrefix = s"$userHome/.m2/repository"
    val tesnsorflowV = "0.1.1-SNAPSHOT"
    val scalaV = "2.11"
    val pathToJar = s"$userHome/.coursier/cache/v1/https/oss.sonatype.org/content/repositories/snapshots/org/platanios/tensorflow_$scalaV/$tesnsorflowV/tensorflow_$scalaV-$tesnsorflowV-linux-cpu-x86_64.jar"
    interp.load.cp(
      Seq(
        ammonite.ops.Path(pathToJar)
      )
    )
  }


res0: {val userHome: String;val pathPrefix: String;val tesnsorflowV: String;val scalaV: String;val pathToJar: String} = $sess.cmd0Wrapper$Helper$$anon$1@7180449c

In [2]:
    import ammonite._, Resolvers._

    val ossSnapshots = Resolver.Http(
      "oss-snapshots",
      "https://oss.sonatype.org/content/repositories/snapshots", //trailing slash!
      IvyPattern,
      true
    )

    interp.resolvers() = interp.resolvers() :+ ossSnapshots

import ammonite._, Resolvers._

    
ossSnapshots: Resolver.Http = Http(
  "oss-snapshots",
  "https://oss.sonatype.org/content/repositories/snapshots",
  "[organisation]/[module]/(scala_[scalaVersion]/)(sbt_[sbtVersion]/)[revision]/[type]s/[artifact](-[classifier]).[ext]",
  true,
  None
)

In [5]:
interp

res4: runtime.InterpAPI = ammonite.interp.Interpreter$$anon$2@67002d6a

In [4]:
//boilerplate
import $ivy.`ml-worthing::introtoml:0.1-SNAPSHOT`

// import $ivy.`org.platanios::tensorflow:0.1.1-SNAPSHOT`

import org.platanios.tensorflow.api
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.core.client.FeedMap

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._
import plotly.element.Orientation._


plotly.JupyterScala.init()


import $ivy.$                                    

// import $ivy.`org.platanios::tensorflow:0.1.1-SNAPSHOT`


import org.platanios.tensorflow.api

import org.platanios.tensorflow.api._

import org.platanios.tensorflow.api.core.client.FeedMap


import plotly._

import plotly.element._

import plotly.layout._

import plotly.JupyterScala._

import plotly.element.Orientation._




In [5]:
  val initialX = 20.0f
  val x: Variable = tf.variable("x", FLOAT32, Shape(1), tf.ConstantInitializer(initialX))

  val x0: Tensor = Tensor(FLOAT32, 1)
  val x1: Output = x
  val x2: Output = x * x
  val x3: Output = x * x * x
  val x4: Output = x * x * x * x

  val xTensor: Output = tf.concatenate(Seq(x0, x1, x2, x3, x4))

  val weights: Tensor = Tensor(FLOAT32, 1, 2, 3, -4.1, 1)

  val loss: Output = tf.sum(xTensor * weights) // any ideas how to improve this using tf.matmul or dot?
  val learningRate = 0.001
  val opt: Op = tf.train.GradientDescent(learningRate).minimize(loss)


2018-02-04 15:43:57.376 [pool-6-thread-13] INFO  TensorFlow Native - Extracting the 'tensorflow_framework' native library to /tmp/tensorflow_scala_native_libraries8751959767378356980/libtensorflow_framework.so.
2018-02-04 15:43:57.466 [pool-6-thread-13] INFO  TensorFlow Native - Copied 15643920 bytes to /tmp/tensorflow_scala_native_libraries8751959767378356980/libtensorflow_framework.so.
2018-02-04 15:43:57.467 [pool-6-thread-13] INFO  TensorFlow Native - Extracting the 'tensorflow' native library to /tmp/tensorflow_scala_native_libraries8751959767378356980/libtensorflow.so.
2018-02-04 15:43:57.700 [pool-6-thread-13] INFO  TensorFlow Native - Copied 51510192 bytes to /tmp/tensorflow_scala_native_libraries8751959767378356980/libtensorflow.so.
2018-02-04 15:43:57.701 [pool-6-thread-13] INFO  TensorFlow Native - Extracting the 'tensorflow_jni' native library to /tmp/tensorflow_scala_native_libraries8751959767378356980/libtensorflow_jni.so.
2018-02-04 15:43:57.705 [pool-6-thread-13] INFO  

initialX: Float = 20.0F
x: Variable = x
x0: Tensor = FLOAT32[1]
x1: Output = Output(name = x/ReadVariable:0, shape = [1], dataType = FLOAT32)
x2: Output = Output(name = Mul:0, shape = [1], dataType = FLOAT32)
x3: Output = Output(name = Mul_2:0, shape = [1], dataType = FLOAT32)
x4: Output = Output(name = Mul_5:0, shape = [1], dataType = FLOAT32)
xTensor: Output = Output(name = Concatenate:0, shape = [5], dataType = FLOAT32)
weights: Tensor = FLOAT32[5]
loss: Output = Output(name = Sum:0, shape = [], dataType = FLOAT32)
learningRate: Double = 0.001
opt: Op = Minimize_1/Minimize

In [7]:
  val session = Session()
  session.run(targets = tf.globalVariablesInitializer())


session: Session = org.platanios.tensorflow.api.core.client.Session@78a7586f
res6_1: Seq[core.client.Fetchable.outputFetchable.ResultType] = List()

In [8]:
  (0 to 100).foreach { iteration =>
    val result: Seq[Tensor] = session.run(fetches = Seq(loss, x1), targets = opt)
    //    result.foreach(tensor => println(tensor.summarize(flattened = true)))
    println(iteration, result(0).scalar, result(1).scalar)
    //    val s@Seq(resultLoss, resultX) = session.run(feeds = feedMap, Seq(loss, x), opt)
    //    println(iteration, resultX.scalar, resultLoss.scalar)
  }

(0,128441.0,20.0)
(1,4364.171,-7.2020016)
(2,1227.542,-5.0285654)
(3,650.18256,-4.180752)
(4,410.67267,-3.6503832)
(5,284.81787,-3.27201)
(6,209.63815,-2.9825723)
(7,160.8618,-2.7511306)
(8,127.31546,-2.5602384)
(9,103.214836,-2.399125)
(10,85.30162,-2.2606983)
(11,71.618935,-2.1400564)
(12,60.93056,-2.0336795)
(13,52.422882,-1.9389625)
(14,45.54191,-1.8539273)
(15,39.899494,-1.7770399)
(16,35.216923,-1.7070892)
(17,31.289768,-1.6431037)
(18,27.965279,-1.5842936)
(19,25.12742,-1.5300088)
(20,22.686777,-1.4797088)
(21,20.573517,-1.4329396)
(22,18.732471,-1.3893172)
(23,17.119577,-1.3485131)
(24,15.6993065,-1.3102456)
(25,14.442753,-1.2742708)
(26,13.326211,-1.2403764)
(27,12.330083,-1.2083766)
(28,11.4380665,-1.1781085)
(29,10.636502,-1.1494275)
(30,9.913888,-1.122206)
(31,9.260474,-1.0963298)
(32,8.667954,-1.0716971)
(33,8.129226,-1.0482165)
(34,7.6381745,-1.0258055)
(35,7.1895256,-1.0043899)
(36,6.7787046,-0.9839024)
(37,6.401728,-0.96428186)
(38,6.0551205,-0.94547266)
(39,5.7358294,-